평당월세 계산 ("KIC_price.csv"파일 있으면 flow 시작가능)

In [ ]:
import pandas as pd

df = pd.read_csv('KIC_price.csv')

평당월세 = (df.iloc[:, 3] / df.iloc[:, 6]).round(2) 

df.insert(5, '평당월세', 평당월세)

df.to_csv('KIC_price_1.csv', index=False, encoding='utf-8-sig')

print("평당월세 데이터를 계산하여 KIC_price_1.csv로 저장했습니다.")

평당월세 데이터를 계산하여 KIC_price_1.csv로 저장했습니다.


층데이터 나누기 

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('KIC_price_1.csv')

df.iloc[:, 8] = df.iloc[:, 8].astype(str)

층_정보 = df.iloc[:, 8].str.split('/', expand=True)

if 층_정보.shape[1] == 1:
    층_정보.columns = ['층']
    층_정보['전체층'] = np.nan
else:
    층_정보.columns = ['층', '전체층']

# 층수 열 제거
df = df.drop(df.columns[8], axis=1)

# "층"과 "전체층" 열 삽입
df.insert(8, '층', 층_정보['층'])
df.insert(9, '전체층', 층_정보['전체층'])

# 결과 저장
df.to_csv('KIC_price_2.csv', index=False, encoding='utf-8-sig')

print("층수 열을 '층'과 '전체층'으로 나누어 KIC_price_2.csv로 저장했습니다.")

층수 열을 '층'과 '전체층'으로 나누어 KIC_price_2.csv로 저장했습니다.


In [ ]:
import pandas as pd
import requests
import time
df = pd.read_csv('KIC_price_2.csv')
REST_API_KEY = 'f32e9311c06e941799af2d38345741de'
location_cache = {}

def reverse_geocode(lon, lat):
    # 위도/경도 쌍을 키로 사용하여 이미 조회한 결과인지 확인
    location_key = f"{lon}_{lat}"
    
    # 이미 조회한 위치라면 캐시에서 결과 반환
    if location_key in location_cache:
        print(f"캐시에서 주소 조회: {location_key}")
        return location_cache[location_key]
    
    # 새로운 위치는 API 호출
    print(f"API 호출: {location_key}")
    url = 'https://dapi.kakao.com/v2/local/geo/coord2address.json'
    headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
    params = {'x': lon, 'y': lat}  # x:경도, y:위도
    
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                address = result['documents'][0]['address']
                road_address = result['documents'][0]['road_address']
                
                if road_address:
                    address_result = road_address['address_name']  # 도로명 주소
                else:
                    address_result = address['address_name']  # 지번 주소
                
                # 캐시에 결과 저장
                location_cache[location_key] = address_result
                return address_result
            else:
                location_cache[location_key] = '주소 정보 없음'
                return '주소 정보 없음'
        else:
            return f'오류 발생: {response.status_code}'
    except Exception as e:
        return f'오류 발생: {str(e)}'
    finally:
        time.sleep(0.5)











# 위도/경도 열이 있는지 확인
if df.shape[1] >= 14:  # 최소 14열 이상이어야 13열(경도) 접근 가능
    # 주소 열 추가
    df['주소'] = ''
    
    # 위도/경도로 주소 찾기
    for idx, row in df.iterrows():
        lon = row.iloc[12]  # 13열: 경도
        lat = row.iloc[11]  # 12열: 위도
        
        # 위도/경도가 유효한 값인지 확인
        if pd.notna(lon) and pd.notna(lat):
            address = reverse_geocode(lon, lat)
            df.loc[idx, '주소'] = address
        else:
            df.loc[idx, '주소'] = '좌표 정보 없음'
        
        # 진행상황 표시 (10개마다)
        if idx % 10 == 0:
            print(f"처리 중: {idx}/{len(df)}")
    
    # 결과 저장
    df.to_csv('KIC_price_3.csv', index=False, encoding='utf-8-sig')
    print("주소 정보를 추가하여 KIC_price_3.csv로 저장했습니다.")
    
    # API 호출 횟수와 캐시 사용 횟수 출력
    unique_locations = len(location_cache)
    total_locations = len(df)
    print(f"전체 위치 수: {total_locations}")
    print(f"고유 위치 수: {unique_locations}")
    print(f"API 호출 횟수: {unique_locations}")
    print(f"캐시 사용 횟수: {total_locations - unique_locations}")
else:
    print("데이터프레임에 위도/경도 열이 없습니다.")

데이터프레임에 위도/경도 열이 없습니다.


도로명주소 찾기

In [10]:
import pandas as pd
import requests
import time

# 데이터 읽기
df = pd.read_csv('KIC_price_2.csv')

# Kakao REST API Key
REST_API_KEY = 'f32e9311c06e941799af2d38345741de'
location_cache = {}

def reverse_geocode(lon, lat):
    """위도/경도를 기반으로 지번 주소를 조회하는 함수"""
    location_key = f"{lon}_{lat}"
    
    # 캐시 확인
    if location_key in location_cache:
        return location_cache[location_key]
    
    # API 호출
    url = 'https://dapi.kakao.com/v2/local/geo/coord2address.json'
    headers = {'Authorization': f'KakaoAK {REST_API_KEY}'}
    params = {'x': lon, 'y': lat}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            result = response.json()
            if result['documents']:
                address = result['documents'][0].get('address')
                address_result = address['address_name'] if address else '주소 정보 없음'
                location_cache[location_key] = address_result
                return address_result
            else:
                location_cache[location_key] = '주소 정보 없음'
                return '주소 정보 없음'
        else:
            return f'오류 발생: {response.status_code}'
    except Exception as e:
        return f'오류 발생: {str(e)}'
    finally:
        time.sleep(0.5)

# 위도/경도 열이 있는지 확인
if '위도' in df.columns and '경도' in df.columns:
    df['지번주소'] = ''  # 새로운 "지번주소" 열 추가
    
    for idx, row in df.iterrows():
        lon = row['경도']  # 경도
        lat = row['위도']  # 위도
        
        if pd.notna(lon) and pd.notna(lat):
            address = reverse_geocode(lon, lat)
            df.loc[idx, '지번주소'] = address
        else:
            df.loc[idx, '지번주소'] = '좌표 정보 없음'
        
        if idx % 10 == 0:
            print(f"처리 중: {idx}/{len(df)}")
    
    # 결과 저장
    df.to_csv('KIC_price_지번주소.csv', index=False, encoding='utf-8-sig')
    print("지번 주소 정보를 추가하여 KIC_price_지번주소.csv로 저장했습니다.")
    
    # 캐시 및 API 호출 정보 출력
    unique_locations = len(location_cache)
    total_locations = len(df)
    print(f"전체 위치 수: {total_locations}")
    print(f"고유 위치 수: {unique_locations}")
    print(f"API 호출 횟수: {unique_locations}")
    print(f"캐시 사용 횟수: {total_locations - unique_locations}")
else:
    print("데이터프레임에 '위도'와 '경도' 열이 없습니다.")

처리 중: 0/1349
처리 중: 10/1349
처리 중: 20/1349
처리 중: 30/1349
처리 중: 40/1349
처리 중: 50/1349
처리 중: 60/1349
처리 중: 70/1349
처리 중: 80/1349
처리 중: 90/1349
처리 중: 100/1349
처리 중: 110/1349
처리 중: 120/1349
처리 중: 130/1349
처리 중: 140/1349
처리 중: 150/1349
처리 중: 160/1349
처리 중: 170/1349
처리 중: 180/1349
처리 중: 190/1349
처리 중: 200/1349
처리 중: 210/1349
처리 중: 220/1349
처리 중: 230/1349
처리 중: 240/1349
처리 중: 250/1349
처리 중: 260/1349
처리 중: 270/1349
처리 중: 280/1349
처리 중: 290/1349
처리 중: 300/1349
처리 중: 310/1349
처리 중: 320/1349
처리 중: 330/1349
처리 중: 340/1349
처리 중: 350/1349
처리 중: 360/1349
처리 중: 370/1349
처리 중: 380/1349
처리 중: 390/1349
처리 중: 400/1349
처리 중: 410/1349
처리 중: 420/1349
처리 중: 430/1349
처리 중: 440/1349
처리 중: 450/1349
처리 중: 460/1349
처리 중: 470/1349
처리 중: 480/1349
처리 중: 490/1349
처리 중: 500/1349
처리 중: 510/1349
처리 중: 520/1349
처리 중: 530/1349
처리 중: 540/1349
처리 중: 550/1349
처리 중: 560/1349
처리 중: 570/1349
처리 중: 580/1349
처리 중: 590/1349
처리 중: 600/1349
처리 중: 610/1349
처리 중: 620/1349
처리 중: 630/1349
처리 중: 640/1349
처리 중: 650/1349
처리 중: 660/1349
처리 중: 

"지번주소"열과 "기타정보"열의 위치를 바꾸기


In [13]:
import pandas as pd

# 데이터 읽기
df = pd.read_csv('KIC_price_3.csv')

# 열 이름 확인
if '지번주소' in df.columns and '기타정보' in df.columns:
    # 열 위치 변경
    columns = list(df.columns)
    address_index = columns.index('지번주소')
    other_info_index = columns.index('기타정보')
    
    # "주소"와 "기타정보" 열 위치를 교환
    columns[address_index], columns[other_info_index] = columns[other_info_index], columns[address_index]
    df = df[columns]
    
    # 결과 저장
    df.to_csv('KIC_price_4.csv', index=False, encoding='utf-8-sig')
    print("주소 열과 기타정보 열의 위치를 바꾸어 KIC_price_4.csv로 저장했습니다.")
else:
    print("'주소' 또는 '기타정보' 열이 데이터프레임에 없습니다.")

주소 열과 기타정보 열의 위치를 바꾸어 KIC_price_4.csv로 저장했습니다.
